In [1]:
import ee


In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0QAtveoVhgszCvGViomhpQxuhhK1nshD8tD4BAtlHDE&tc=dCJV_sJsg0fEssY8vitjf-EYKB681jsRy1i2HTFpmX4&cc=-wcc-B0m8pSU9WOQjXosHDlMMd_15bMyFIz-bXHIrmM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7ahzP7V9bqFhHe3VrwLoJUX66GXQ3JcjD1rMNGQHW5vftG9IsAD0E

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [2]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.4 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=213efa772d29355e75c9da546a65b98d5d6e720eb0f879c57c6a1

In [3]:
import geemap

In [4]:
Map = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=1xw44ajKPxiebmVv5tAljM-SZUpIiBAbkd-jPGOCc1Y&tc=NgtB4TJXO0fmVcnKRroZPLNn2CGv80zYyNfWFwYcRO8&cc=P2JFpZrPHHmcxUUh1VbiYEi2zLI_JN-aK5o5FI3hCiE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4JNi4KLIpSVhTSlN84ePQTJh2Iy-i4KRHD0qmM7CiBndeF4U7fZW8

Successfully saved authorization token.


In [5]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import folium

In [7]:
#roi = ee.Geometry.Polygon(
 # [[[85.14,19.37],
  #  [85.14,19.54],
   # [84.95,19.54],
    #[84.95,19.37]]])
admin2 = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
Orissa = admin2.filter(ee.Filter.eq('ADM1_NAME', 'Orissa'))
roi = Orissa.filter(ee.Filter.eq('ADM2_NAME','Ganjam'))

Map.centerObject(roi, 10)
img = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

filtered = img \
  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
  .filter(ee.Filter.bounds(roi)) \

def cloud_mask_l8(image):
    # Select the pixel QA band
    qa = image.select('pixel_qa')
    # Make a mask of clear and water pixels
    clear = qa.bitwiseAnd(int('0000000000000010', 2)).neq(0)
    water = qa.bitwiseAnd(int('0000000000000100', 2)).neq(0)
    # Mask out cloudy or shadowed pixels
    masked = image.updateMask(clear.Or(water))
    return masked

cloud_masked=filtered.map(cloud_mask_l8)

composite = cloud_masked.median()
mosaic = cloud_masked.mosaic()

palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']

vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
    
}

Map.addLayer(composite.clip(roi),vis_params ,'vis')
Map



Map(center=[19.62068846825679, 84.658719741119], controls=(WidgetControl(options=['position', 'transparent_bg'…

In [8]:
#import training data
training = ee.FeatureCollection('projects/ee-sharmavinayak4u/assets/assetIDLULC')

label = "lulc"
bands = ['B1','B2','B3','B4','B5','B6','B7']
input = composite.clip(roi).select(bands)


In [9]:
#overlay the points i=on the imagery to get training
trainImage= input.sampleRegions(**{
    'collection': training,
    'properties': [label],
    'scale': 30
})

trainingData = trainImage.randomColumn()


In [10]:
#train_test_split
trainSet = trainingData.filter(ee.Filter.lessThan('random',0.8))
testSet = trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.8))
testSet.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '00000000000000000000_0',
 'properties': {'B1': 177,
  'B2': 239,
  'B3': 524,
  'B4': 372.5,
  'B5': 2531.5,
  'B6': 1175,
  'B7': 619.5,
  'lulc': 1,
  'random': 0.8426323457752717}}

In [11]:
#classification model
#classifier = ee.Classifier.smileCart().train(trainSet,label,bands)
#classifiy
#classified = input.classify(classifier)
#print(classified.getInfo())
classifier = ee.Classifier.smileRandomForest(10).train(trainSet,label,bands)
classified = input.select(bands).classify(classifier)

In [12]:
#define paletter
landcoverPalette = [
    '#2850d6',#water                                 #205ad6
    '#6bff1a', #forest                               #75f231
    '#cf1515', #urban                                #ff1717
    '#fff62a', #cropland                             #e1a744
    '#859491', #barren                               #b1c2ae
]

In [13]:
#define a method for displaying ee image tiles to folium map
def add_ee_layer(self,ee_image_object,vis_p,name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_p)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
    ).add_to(self)

In [14]:
#add ee drawing method to folium
folium.Map.add_ee_layer = add_ee_layer
# create a folium map object
my_map = folium.Map(location=[84.99,19.43],zoom_start = 3)

cl_vis_params= {
    'palette': landcoverPalette,
    'min': 1,
    'max': 5,
        
}
#add layer to the map
combined_lulc=composite.clip(roi).visualize(**vis_params)
combined_classification=classified.visualize(**cl_vis_params)
my_map.add_ee_layer(combined_lulc,{},"landsat 2020")
my_map.add_ee_layer(combined_classification,{},"classification")

#add control panel
my_map.add_child(folium.LayerControl())
display(my_map)

In [15]:

train_accuracy = classifier.confusionMatrix()
train_accuracy.getInfo()


[[0, 0, 0, 0, 0, 0],
 [0, 50, 1, 0, 0, 1],
 [0, 0, 39, 0, 2, 0],
 [0, 0, 0, 48, 2, 0],
 [0, 0, 0, 0, 70, 0],
 [0, 0, 1, 0, 0, 38]]

In [16]:
train_accuracy.accuracy().getInfo()

0.9722222222222222

In [17]:
train_accuracy.kappa().getInfo()

0.9648058736682494

In [18]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9615384615384616],
 [0.9512195121951219],
 [0.96],
 [1],
 [0.9743589743589743]]

In [19]:
train_accuracy.consumersAccuracy().getInfo()

[[0, 1, 0.9512195121951219, 1, 0.9459459459459459, 0.9743589743589743]]

**Validation Accuracy**

In [20]:
validated= trainSet.classify(classifier)
validated.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '00000000000000000001_0',
 'properties': {'B1': 262,
  'B2': 296.5,
  'B3': 421,
  'B4': 255,
  'B5': 477,
  'B6': 156.5,
  'B7': 88,
  'classification': 1,
  'lulc': 1,
  'random': 0.28980225312345786}}

In [21]:
test_accuracy = validated.errorMatrix('lulc','classification')
test_accuracy.getInfo()
test_accuracy.accuracy().getInfo()

0.9722222222222222

In [22]:
test_accuracy.kappa().getInfo()

0.9648058736682494

In [23]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9615384615384616],
 [0.9512195121951219],
 [0.96],
 [1],
 [0.9743589743589743]]

In [24]:
test_accuracy.consumersAccuracy().getInfo()

[[0, 1, 0.9512195121951219, 1, 0.9459459459459459, 0.9743589743589743]]

In [ ]:
geometry = roi.geometry()  # Define your region of interest

export_params = {
    'image': combined_classification,
    'description': 'classification_RF_',
    'bucket': 'lulcbucket',  # Replace with your GCS bucket name
    'fileFormat': 'auto',
    'maxZoom': 13,
    'region': geometry,  # Use the extracted geometry as the export region
    'writePublicTiles': True
}  # Define your export parameters as a dictionary object

task = ee.batch.Export.map.toCloudStorage(**export_params)  # Call the Export.map.toCloudStorage() function with the export parameters
task.start()  # Start the export task

In [ ]:

export_params = {
  'image': combined_classification,
  'description': 'classification_RF_image_asset',
  'assetId': 'projects/ee-sharmavinayak4u/assets/ASSET_ID_LULC_Main',
  'pyramidingPolicy': {
    '.default': 'mode'
  },
  'region': geometry,
  'scale': 30,
  'maxPixels': 1e13
}

# Export the image to an Asset
task = ee.batch.Export.image.toAsset(**export_params)
task.start()

In [ ]:
# Write a function that computes NDVI for an image and adds it as a band
def addNDVI(image):
  ndvi = image.normalizedDifference(['B5', 'B4']).rename('ndvi')
  return image.addBands(ndvi)

withNdvi = filtered.map(addNDVI)

composite = withNdvi.median()

palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
  '74A901', '66A000', '529400', '3E8601', '207401', '056201',
  '004C00', '023B01', '012E01', '011D01', '011301']

ndviVis = {'min':0, 'max':0.5, 'palette': palette }
Map.addLayer(withNdvi.select('ndvi'), ndviVis, 'NDVI Composite')
Map

In [ ]:
def maskS2clouds(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask) \
      .select("B.*") \
      .copyProperties(image, ["system:time_start"])
